In [5]:
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import chain
import numpy as np

# Read in the datasets
left_df = pd.read_csv("left_dataset.csv")
right_df = pd.read_csv("right_dataset.csv")

# Make copies of the original dataframes for cleaning
left_df_cleaned = left_df.copy()
right_df_cleaned = right_df.copy()

# Replace "N/A" values in the "address" column with NaN
left_df_cleaned['address'] = left_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)
right_df_cleaned['address'] = right_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)

# Remove trailing commas from the "address" column
left_df_cleaned['address'] = left_df_cleaned['address'].str.rstrip(',')
right_df_cleaned['address'] = right_df_cleaned['address'].str.rstrip(',')

# Convert strings to lowercase/uppercase
left_df_cleaned['address'] = left_df_cleaned['address'].str.lower()
right_df_cleaned['address'] = right_df_cleaned['address'].str.lower()
left_df_cleaned['name'] = left_df_cleaned['name'].str.lower()
right_df_cleaned['name'] = right_df_cleaned['name'].str.lower()
left_df_cleaned['city'] = left_df_cleaned['city'].str.lower()
right_df_cleaned['city'] = right_df_cleaned['city'].str.lower()
left_df_cleaned['state'] = left_df_cleaned['state'].str.upper()
right_df_cleaned['state'] = right_df_cleaned['state'].str.upper()

## clean zip codes
# clean zip code in left dataset
left_df_cleaned['zip_code'] = left_df_cleaned['zip_code'].str.slice(0, 5)

# clean zip code in right dataset
right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].fillna('')
right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].astype(str)
right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].str.replace(r'\.\d+', '')

#
left_df_cleaned = left_df_cleaned.drop_duplicates(subset = ['name','address'],keep='first')
right_df_cleaned = right_df_cleaned.drop_duplicates(subset = ['name','address'],keep='first')

C:\Users\mikes\AppData\Local\Temp\ipykernel_10328\271406851.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  left_df_cleaned['address'] = left_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)
C:\Users\mikes\AppData\Local\Temp\ipykernel_10328\271406851.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  right_df_cleaned['address'] = right_df_cleaned['address'].str.replace(r'\bN/A\b', '').replace('', pd.NA)
C:\Users\mikes\AppData\Local\Temp\ipykernel_10328\271406851.py:39: FutureWarning: The default value of regex will change from True to False in a future version.
  right_df_cleaned['postal_code'] = right_df_cleaned['postal_code'].str.replace(r'\.\d+', '')


In [11]:
# Function used to find matches in two small dataframes
def matcher(left_df, right_df):
    
    matches = []
    for _, lrow in left_df.iterrows():
        for _, rrow in right_df.iterrows():
            score1 = fuzz.token_set_ratio(lrow['name'], rrow['name']) / 100
            score2 = fuzz.token_set_ratio(lrow['address'], rrow['address']) / 100
            
            if min(score1,score2)>= 0.8:
                matches.append([lrow['business_id'],rrow['entity_id'],min(score1,score2)])
    return matches

In [16]:
# Function used to find matches in two big dataframes by clustering them based on postal code
def approximate_match(left_df,right_df):
    postal_codes = set(left_df['zip_code'])&set(right_df['postal_code'])
    result=[]
    
    left_group = left_df.groupby('zip_code', as_index = True)
    right_group = right_df.groupby('postal_code', as_index = True)
    
    for i in postal_codes:
        left = left_group.get_group(i)
        right = right_group.get_group(i)
        matches = matcher(left,right)
        result.extend(matches)
    return result

In [28]:
result = approximate_match(left_df_cleaned,right_df_cleaned)

In [39]:
output = pd.DataFrame(result,columns = ['left_dataset','right_dataset','confidence_score'])

In [40]:
output

,left_dataset,right_dataset,confidence_score
0,63128,73265,1.00
1,63262,54860,0.91
2,65886,54860,1.00
3,86289,38606,1.00
4,87053,40400,1.00
...,...,...,...
9924,96932,22813,1.00
9925,97369,72330,1.00
9926,97751,80443,1.00
9927,97944,25031,1.00


In [46]:
output.to_csv('matches.csv',index=False)